Crawler para extração dos itens do carrinho da Atividade Prática.

# Imports

In [1]:
import requests, json, threading, sys, traceback
import pandas as pd

# Funções

In [118]:
def get_category(category: str) -> list:
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    payload = {
        "partner": "linx",
        "page": 1,
        "resultsPerPage": 300,
        "multiCategory": category,
        "sortBy": "relevance",
        "department": "ecom",
        "storeId": 461,
        "customerPlus": True
    }
    
    try:
        # Pega o número total de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        max_produtos = response.json()['totalProducts']
        payload['resultsPerPage'] = max_produtos

        # Pega a lista de produtos da categoria
        response = requests.post(r"https://api.vendas.gpa.digital/pa/search/category-page", headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        
        lista_p = list(response.json()['products'])
        lista_cod = []

        for prod in lista_p:
            lista_cod.append(prod['id'])
            
    except:
        return []
    
    else:
        return lista_cod

In [2]:
def get_product(code: int, verbose: bool = False) -> dict:
    try:
        # Fazendo a requisição
        texto = requests.get(fr"https://www.paodeacucar.com/produto/{code}").text
        
        # Encontrando as posições
        inicio = texto.find('"props":') + len('"props":')
        fim = texto.find(',"page":"/produto"', inicio)

        # Extraindo a substring
        substring = texto[inicio:fim].strip()
        
        # Dicionario Geral
        dicionario = json.loads(substring)
        
        # Informações gerais do produto
        informacoes = dicionario['initialProps']['componentProps']['product']
        atributos = get_dictkey(informacoes, 'attributeGroups')
        
        # Dicionário com os resultados
        resultados = {    
        'id': informacoes['id'], 
        'nome': informacoes['name'], 
        'sku': informacoes['sku'], 
        'descricao': informacoes['description'],
        'imagem': f'https://static.paodeacucar.com{informacoes['thumbPath']}',
        'preco': get_dictkey(informacoes, 'currentPrice'),
        'marca': get_dictcode(atributos, 'Marca'),
        'alturaProduto': get_dictcode(atributos, 'alturaProduto'),
        'larguraProduto': get_dictcode(atributos, 'larguraProduto'),
        'pesoBruto': get_dictcode(atributos, 'pesoBruto'),
        'unidadePesoBruto': get_dictcode(atributos, 'unidadePesoBruto'),
        'profundidadeProduto': get_dictcode(atributos, 'profundidadeProduto')
        }
        
        assert(
            resultados['id'] == code and 
            resultados['preco'] != None and
            resultados['alturaProduto'] != None and 
            resultados['larguraProduto'] != None and 
            resultados['pesoBruto'] != None and 
            resultados['unidadePesoBruto'] != None and
            resultados['profundidadeProduto'] != None
        )

    except:
        if verbose:
            exc_info = sys.exc_info()
            e = ''.join(traceback.format_exception(*exc_info))
            del exc_info
            
            print(f"Erro ao extrair informações do produto {code}!\nErro:\n{e}")
            
            if 'informacoes' in locals():
                print(json.dumps(locals()['informacoes'], indent=2))
                
            if 'informacoes2' in locals():
                print(json.dumps(locals()['informacoes2'], indent=2))
        
        return None
    
    else:
        return resultados

In [120]:
def fetch_products(codigos, verbose: bool = False):
    resultados = []
    threads = []
    lock = threading.Lock()

    def thread_function(codigo, verbose):
        info = get_product(codigo, verbose)
        
        if info:
            with lock:
                resultados.append(info)

    for codigo in codigos:
        thread = threading.Thread(target=thread_function, args=(codigo, verbose,))
        threads.append(thread)
        thread.start()

    while any(thread.is_alive() for thread in threads): pass

    return resultados


In [121]:
def get_dictkey(dicionario, chave):
    # Função interna recursiva
    def busca_recursiva(d):
        if isinstance(d, dict):
            for k, v in d.items():
                if k == chave:
                    return v
                resultado = busca_recursiva(v)
                if resultado is not None:
                    return resultado
                
        elif isinstance(d, list):
            for item in d:
                resultado = busca_recursiva(item)
                if resultado is not None:
                    return resultado
        return None

    return busca_recursiva(dicionario)

In [122]:
def get_dictcode(d, code):
    # Se d for um dicionário, iteramos sobre suas chaves e valores
    if isinstance(d, dict):
        if d.get('code') == code:
            return d.get('value')  # Retorna o valor se a chave 'code' for encontrada com o valor especificado
        for value in d.values():
            result = find_value_by_code(value, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    # Se d for uma lista, iteramos sobre os elementos da lista
    elif isinstance(d, list):
        for item in d:
            result = find_value_by_code(item, code)  # Chamada recursiva
            if result is not None:
                return result  # Retorna o resultado se o dicionário for encontrado

    return None  # Retorna None se o dicionário não for encontrado

In [123]:
def format_desc(x):
    for rep in ['<b>', '</b>', '<br>', '</br>', '<p>', '</p>', '\r', '\n', r'"""¿']:
        x = x.replace(rep, '')
    
    return x

# Constants

In [124]:
CATEGORIAS = ['alimentos', 'bebidas', 'limpeza', 'petshop', 'bebidas-alcoolicas-', 'bebes-e-criancas', 'cuidados-pessoais', 'suplementos-alimentares', 'floricultura-e-jardim']

# Code

In [ ]:
map = []

for category in CATEGORIAS:
    print(f"Obtendo códigos de produtos da categoria {category}...")
    codigos = get_category(category)
    
    print(f"Obtendo informações dos produtos da categoria {category}...")
    # map[category] = fetch_products(codigos)
    for item in fetch_products(codigos):
        map.append({**item, **{'categoria': category}})

map

In [ ]:
df = pd.concat([pd.DataFrame([item]) for item in map], ignore_index=True, sort=False, axis='index')
df

In [ ]:
df['descricao'] = df['descricao'].apply(format_desc)
df['descricao'] = df['descricao'].fillna('Sem descrição')
df['pesoBruto'] = df['pesoBruto'].astype(float)
df['pesoBruto'] = df.apply(lambda x: x['pesoBruto'] if x['unidadePesoBruto'] == 'G' else x['pesoBruto'] * 1000, axis=1)
df.drop(columns=['unidadePesoBruto'], inplace=True)
df

In [128]:
df.to_csv(r'..\data\produtos.csv', index=False, sep='|', encoding='utf-8')